## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-30-01-09-21 +0000,nyt,Middle East Updates: Trump Releases Gaza Plan ...,https://www.nytimes.com/live/2025/09/29/world/...
1,2025-09-30-01-06-55 +0000,wapo,Judge blocks Kari Lake from laying off over 50...,https://www.washingtonpost.com/business/2025/0...
2,2025-09-30-01-00-00 +0000,wsj,Why General Motors Boss Mary Barra Is Slamming...,https://www.wsj.com/business/autos/general-mot...
3,2025-09-30-01-00-00 +0000,wsj,This Windowless Plane Is Vying to Be the Priva...,https://www.wsj.com/business/airlines/this-win...
4,2025-09-30-01-00-00 +0000,wsj,Boeing Has Started Working on a 737 MAX Replac...,https://www.wsj.com/business/airlines/boeing-h...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2417/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
3,trump,58
122,new,18
201,government,16
5,gaza,15
202,shutdown,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
89,2025-09-29-21-14-29 +0000,nypost,Trump unveils 20-point Gaza cease-fire plan wi...,https://nypost.com/2025/09/29/us-news/trump-un...,143
27,2025-09-30-00-00-00 +0000,wsj,Top Democrats made their case to Trump for res...,https://www.wsj.com/politics/policy/government...,129
53,2025-09-29-22-29-55 +0000,nypost,Ex-UK PM Tony Blair to help run Gaza under Tru...,https://nypost.com/2025/09/29/us-news/ex-uk-pm...,119
21,2025-09-30-00-15-32 +0000,bbc,Trump and Netanyahu agree new Gaza peace plan,https://www.bbc.com/news/articles/c1dq9xwngv2o...,118
313,2025-09-29-01-58-54 +0000,nypost,President Trump shows off new White House blin...,https://nypost.com/2025/09/28/us-news/presiden...,117


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
89,143,2025-09-29-21-14-29 +0000,nypost,Trump unveils 20-point Gaza cease-fire plan wi...,https://nypost.com/2025/09/29/us-news/trump-un...
27,71,2025-09-30-00-00-00 +0000,wsj,Top Democrats made their case to Trump for res...,https://www.wsj.com/politics/policy/government...
290,63,2025-09-29-09-00-00 +0000,wsj,"Rahm Emanuel, former Democratic congressman, W...",https://www.wsj.com/politics/policy/rahm-emanu...
132,53,2025-09-29-18-55-45 +0000,nypost,Frank founder Charlie Javice sobs in court as ...,https://nypost.com/2025/09/29/business/charlie...
75,52,2025-09-29-21-42-18 +0000,nypost,New Yorkers ‘not safe’ with a Zohran Mamdani o...,https://nypost.com/2025/09/29/us-news/new-york...
169,49,2025-09-29-16-49-04 +0000,cbc,Police continue to search for motive in deadly...,https://www.cbc.ca/news/canada/windsor/deadly-...
268,39,2025-09-29-10-44-08 +0000,cbc,Moldova's pro-EU party wins election after cam...,https://www.cbc.ca/news/world/moldova-parliame...
121,39,2025-09-29-19-32-00 +0000,wsj,The clock is ticking down to an Oct. 1 deadlin...,https://www.wsj.com/politics/policy/government...
279,37,2025-09-29-09-30-00 +0000,wsj,"The Pentagon, alarmed at the low weapons stock...",https://www.wsj.com/politics/national-security...
100,36,2025-09-29-20-53-00 +0000,wsj,The Trump administration said it is cracking d...,https://www.wsj.com/politics/national-security...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
